In [1]:
import os
import pandas as pd
import numpy as np
import time
from datetime import date
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from lightgbm import LGBMRegressor
from sklearn.utils import resample
import catboost as cat
from catboost import Pool
# import pyfolio as pf



import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.chdir(r'e:/jiaqifiles/Rdata')

for actdays in np.arange(1,2):
    input_filename = 'SHSZ_'+ str(actdays) +'.feather'
    print(input_filename)
    
    MLdata = pd.read_feather(input_filename) # the data is w/o index, the data is already Z scored on classy and fillna with 0 for x

    print(MLdata.columns)
    ###########################################
    # x_indexs = np.r_[9:56] # M:zscore  10:56 in the r convention
    x_indexs = np.r_[4:94] # M:zscore  5:94 in the r convention
    print(MLdata.columns[x_indexs])
    y_name = "classy"

    # create a list to save all the pred_y and concat them together
    pred_y_list = []

    for y in np.arange(2016,2025):
        time1 = date.fromisoformat(str(y)+'-01-01')
        time2 = date.fromisoformat(str(y+1)+'-01-01')

        train = MLdata[MLdata.Date<time1]
        test = MLdata[(MLdata.Date>=time1) & (MLdata.Date<time2)]

        Ytrain = train[y_name].values
        Xtrain = train.iloc[:,x_indexs].values
    
        # train the default model

        model = cat.CatBoostRegressor(
            logging_level="Silent",
            iterations=5000,
            task_type = 'GPU'
         )

        train_pool = Pool(Xtrain, Ytrain)
        model.fit(train_pool)
    
        # go with the test set
        Xtest = test.iloc[:,x_indexs].values
        Ytest = test[y_name].values
        test_pool = Pool(Xtest, Ytest)   
        Z = model.predict(test_pool)

        pred_y = Z
        bigtest = test.copy() # just my naming convention to get the pred_y into the table
        bigtest['pred_y'] = pred_y
        pred_y_list.append(bigtest)

    bigtest = pd.concat(pred_y_list)

    filename = 'catboost' + input_filename
    bigtest.reset_index().to_feather(filename) # bridge to R





    

SHSZ_1.feather
Index(['ID', 'Date', 'y', 'classy', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7',
       'm8', 'm9', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15', 'm16', 'm17',
       'm18', 'm19', 'm20', 'm21', 'm22', 'm23', 'm24', 'm25', 'm26', 'm27',
       'm28', 'm29', 'm30', 'm31', 'm32', 'm33', 'm34', 'm35', 'm36', 'm37',
       'm38', 'm39', 'm40', 'm41', 'm42', 'm43', 'm44', 'm45', 'm46', 'm47',
       'm48', 'm49', 'm50', 'm51', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7',
       'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'd16', 'd17',
       'd18', 'd19', 'd20', 'tempsd', 'tempskew', 'tempkurt', 'tempsharpe',
       'tempalpha', 'tempbeta', 'tempcorr', 'temp_short_vol', 'tempMax',
       'tempsd2', 'tempskew2', 'tempkurt2', 'tempsharpe2', 'tempalpha2',
       'tempbeta2', 'tempcorr2', 'temp_short_vol2', 'tempMax2', 'Janind'],
      dtype='object')
Index(['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11',
       'm12', 'm13', 'm14', 'm15', 'm16', 'm17', 'm18',